In [16]:
import pandas as pd

# Load CSV files
match_results_df = pd.read_csv('csv/wedstrijden_onverwerkd.csv')
goal_events_df = pd.read_csv('csv/doelpunten_onverwerkd.csv')
standings_df = pd.read_csv('csv/klassement_onverwerkd.csv')

goal_events_df = goal_events_df[goal_events_df['valid_goal'] == True]
# Filter of vervang niet-numerieke waarden
match_results_df['doelpunten_thuisploeg'] = pd.to_numeric(match_results_df['doelpunten_thuisploeg'], errors='coerce').fillna(0)
match_results_df['doelpunten_uitploeg'] = pd.to_numeric(match_results_df['doelpunten_uitploeg'], errors='coerce').fillna(0)

# Nu kunt u de conversie naar int veilig uitvoeren
match_results_df['doelpunten_thuisploeg'] = match_results_df['doelpunten_thuisploeg'].astype(int)
match_results_df['doelpunten_uitploeg'] = match_results_df['doelpunten_uitploeg'].astype(int)

In [17]:
# Vind alle verplaatste matches, namelijk de match id zal dezelfde zijn als de vorige
match_results_df['same_as_previous_match'] = match_results_df['id_match'].eq(match_results_df['id_match'].shift())

# Selecteer alleen de rijen waar 'same_as_previous_match' True is
duplicate_matches = match_results_df[match_results_df['same_as_previous_match']]
duplicate_matches.head(30)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match


In [18]:
# Bereken goals
goals_home_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['thuisploeg_stamnummer']].groupby('id_match').size()
goals_away_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['uitploeg_stamnummer']].groupby('id_match').size()

# Omzetting
goals_home_team_df = goals_home_team.reset_index(name='calculated_doelpunten_thuisploeg')
goals_away_team_df = goals_away_team.reset_index(name='calculated_doelpunten_uitploeg')

# Vervang NaN values in goal kolom met 0
merged_df = pd.merge(match_results_df, goals_home_team_df, on='id_match', how='outer')
merged_df = pd.merge(merged_df, goals_away_team_df, on='id_match', how='outer')
merged_df['calculated_doelpunten_thuisploeg'] = merged_df['calculated_doelpunten_thuisploeg'].fillna(0).astype(int)
merged_df['calculated_doelpunten_uitploeg'] = merged_df['calculated_doelpunten_uitploeg'].fillna(0).astype(int)

# Bereken verschil tussen berekening en resultaat
discrepancies_goals = merged_df[
    (merged_df['calculated_doelpunten_thuisploeg'] != merged_df['doelpunten_thuisploeg']) |
    (merged_df['calculated_doelpunten_uitploeg'] != merged_df['doelpunten_uitploeg'])
]

# Select only relevant columns for display
discrepancies_goals.head(50)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match,calculated_doelpunten_thuisploeg,calculated_doelpunten_uitploeg
356,2022,14,2022/10/23,18:30:00,3851182,16,Standard Luik,35,RSC Anderlecht,5,0,False,3,1
380,2022,17,2022/11/12,20:45:00,3851244,22,R Charleroi SC,25,KV Mechelen,0,5,False,1,0
728,2021,22,2022/01/15,16:15:00,3598295,18,Oud-Heverlee Leuven,25,KV Mechelen,5,0,False,0,0
841,2021,34,2022/04/10,18:30:00,3598351,10,Union Saint-Gilloise,13,Beerschot,5,0,False,0,0
1996,2016,17,2016/12/04,18:00:00,2732129,22,R Charleroi SC,16,Standard Luik,0,0,False,1,3
2427,2014,11,2014/10/19,14:30:00,2503727,16,Standard Luik,5381,SV Zulte Waregem,0,5,False,1,2
2836,2012,2,2012/08/05,18:00:00,2229917,18,Oud-Heverlee Leuven,322,KRC Genk,2,2,False,1,3
3175,2011,15,2011/11/26,18:00:00,1134359,5381,SV Zulte Waregem,35,RSC Anderlecht,2,3,False,0,3
3543,2009,1,2009/08/01,20:00:00,955117,322,KRC Genk,224,Excelsior Mouscron,0,0,False,1,2
3564,2009,3,2009/08/16,18:00:00,958283,19,KV Kortrijk,224,Excelsior Mouscron,0,0,False,2,2


In [19]:
# controle van aantal_wedstrijden
# Geen enkel record met meer wedstrijden dan speeldagen
number_of_games_check = standings_df[standings_df.speeldag < standings_df.aantal_wedstrijden]
number_of_games_check.head(30)

,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten


In [20]:
# controle van aantal_wedstrijden
# wel records met minder wedstrijden dan speeldagen, maar bij nazicht van een aantal records klopt dit
number_of_games_check = standings_df[standings_df.speeldag > standings_df.aantal_wedstrijden]
number_of_games_check.seizoen.value_counts()

1975    551
2002    476
2009    360
Name: seizoen, dtype: int64

In [21]:
# geen records met standings_df.Played != (standings_df.Wins + standings_df.Losses + standings_df.Draws
number_of_games_check = standings_df[standings_df.aantal_wedstrijden != (standings_df.aantal_gewonnen + standings_df.aantal_gelijk + standings_df.aantal_verloren)]
number_of_games_check.count()

seizoen               0
speeldag              0
stand                 0
naam_ploeg            0
id_ploeg              0
aantal_wedstrijden    0
aantal_gewonnen       0
aantal_gelijk         0
aantal_verloren       0
doelpunten_voor       0
doelpunten_tegen      0
verschil              0
punten                0
dtype: int64

In [22]:
# geen records met (uitslagen.doelpunten_voor - uitslagen.doelpunten_tegen) != (uitslagen.verschil)
standings_df['calculated_goal_verschil'] = standings_df['doelpunten_voor'] - standings_df['doelpunten_tegen']
controle_aantal_wedstrijden = standings_df[standings_df['calculated_goal_verschil'] != standings_df['verschil']]
controle_aantal_wedstrijden.count()

seizoen                     0
speeldag                    0
stand                       0
naam_ploeg                  0
id_ploeg                    0
aantal_wedstrijden          0
aantal_gewonnen             0
aantal_gelijk               0
aantal_verloren             0
doelpunten_voor             0
doelpunten_tegen            0
verschil                    0
punten                      0
calculated_goal_verschil    0
dtype: int64

In [23]:
# het totaal aantal gewonnen matchen per seizoen en per ploeg
points_df = standings_df.groupby(['seizoen','speeldag','id_ploeg', 'punten'])[['aantal_gewonnen','aantal_gelijk','aantal_verloren']].sum()
points_df = points_df.reset_index()
points_df = points_df.sort_values(['seizoen','speeldag','aantal_gewonnen','aantal_gelijk','aantal_verloren'], ascending = [True, True, False, False, False])

In [24]:
points_df['punten_check'] = points_df['aantal_gewonnen'] * 3 + points_df['aantal_gelijk'] + points_df['aantal_verloren'] * 0
discrepancies_df = points_df[points_df['punten'] != points_df['punten_check']]
discrepancies_df.head(30)

,seizoen,speeldag,id_ploeg,punten,aantal_gewonnen,aantal_gelijk,aantal_verloren,punten_check
1,1960,1,2,2:0,1,0,0,3
3,1960,1,4,2:0,1,0,0,3
5,1960,1,10,2:0,1,0,0,3
6,1960,1,13,2:0,1,0,0,3
7,1960,1,16,2:0,1,0,0,3
10,1960,1,35,2:0,1,0,0,3
11,1960,1,90,2:0,1,0,0,3
14,1960,1,553,2:0,1,0,0,3
0,1960,1,1,0:2,0,0,1,0
2,1960,1,3,0:2,0,0,1,0


In [25]:
match_results_df['datum'] = pd.to_datetime(match_results_df['datum'])
goal_events_df['datum'] = pd.to_datetime(goal_events_df['datum'])

merged_df = pd.merge(goal_events_df, match_results_df, on='id_match', suffixes=('_goal', '_match'))

# Check for discrepancies in dates
date_discrepancies_df = merged_df[merged_df['datum_goal'] != merged_df['datum_match']]
date_discrepancies_df.head(30)

,seizoen_goal,dag,datum_goal,tijd_goal,id_match,thuisploeg_stamnummer_goal,thuisploeg_goal,uitploeg_goal,uitploeg_stamnummer_goal,goal_time,...,speeldag,datum_match,tijd_match,thuisploeg_stamnummer_match,thuisploeg_match,uitploeg_stamnummer_match,uitploeg_match,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match


In [26]:
# Groepeer de gegevens op 'Day' en controleer de datums
sorted_dates = match_results_df.sort_values(by=['speeldag', 'datum'])
date_discrepancies_df = sorted_dates.groupby('speeldag')['datum'].apply(lambda x: x.is_monotonic_increasing)

date_discrepancies_df = date_discrepancies_df[date_discrepancies_df == False]
date_discrepancies_df.head(30)


Series([], Name: datum, dtype: bool)

In [27]:
# Sorting the data by season, day, and date
sorted_dates = match_results_df.sort_values(by=['seizoen', 'speeldag', 'datum'])

# Group by 'season' and 'day', and check if dates are in chronological order within each group
grouped = sorted_dates.groupby(['seizoen', 'speeldag'])
date_order_check = grouped['datum'].apply(lambda x: x.is_monotonic_increasing)

# Identifying seasons and days where dates are not in chronological order
date_discrepancies = date_order_check[date_order_check == False]
date_discrepancies.head(30)

Series([], Name: datum, dtype: bool)

In [28]:
# Filter matches that ended 0-0
matches_0_0 = match_results_df[(match_results_df['doelpunten_thuisploeg'] == 0) & (match_results_df['doelpunten_uitploeg'] == 0)]

# Get match_ids of 0-0 matches
match_ids_0_0 = matches_0_0['id_match']

# Check if these matches have any goals in goal_events.csv
goals_in_0_0_matches = goal_events_df[goal_events_df['id_match'].isin(match_ids_0_0)]
goals_in_0_0_matches.head(30)

,seizoen,dag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg,uitploeg_stamnummer,goal_time,real_time_goal,goal_team_naam,goal_team_stamnummer,stand_thuis,stand_uit,valid_goal
6434,2016,17,2016-12-04,18:00:00,2732129,22,R Charleroi SC,Standard Luik,16,12,18:12:00,R Charleroi SC,22,1.0,0,True
6435,2016,17,2016-12-04,18:00:00,2732129,22,R Charleroi SC,Standard Luik,16,23,18:23:00,Standard Luik,16,1.0,1,True
6436,2016,17,2016-12-04,18:00:00,2732129,22,R Charleroi SC,Standard Luik,16,51,18:51:00,Standard Luik,16,1.0,2,True
6437,2016,17,2016-12-04,18:00:00,2732129,22,R Charleroi SC,Standard Luik,16,57,18:57:00,Standard Luik,16,1.0,3,True
11341,2009,1,2009-08-01,20:00:00,955117,322,KRC Genk,Excelsior Mouscron,224,4,20:04:00,Excelsior Mouscron,224,0.0,1,True
11342,2009,1,2009-08-01,20:00:00,955117,322,KRC Genk,Excelsior Mouscron,224,77,21:17:00,Excelsior Mouscron,224,0.0,2,True
11343,2009,1,2009-08-01,20:00:00,955117,322,KRC Genk,Excelsior Mouscron,224,91,21:31:00,KRC Genk,322,1.0,2,True
11410,2009,3,2009-08-16,18:00:00,958283,19,KV Kortrijk,Excelsior Mouscron,224,47,18:47:00,Excelsior Mouscron,224,0.0,1,True
11411,2009,3,2009-08-16,18:00:00,958283,19,KV Kortrijk,Excelsior Mouscron,224,70,19:10:00,KV Kortrijk,19,1.0,1,True
11412,2009,3,2009-08-16,18:00:00,958283,19,KV Kortrijk,Excelsior Mouscron,224,73,19:13:00,KV Kortrijk,19,2.0,1,True


In [29]:
# Sorteer de DataFrame op basis van seizoen, speeldag, punten, aantal gewonnen wedstrijden, doelpuntenverschil en gemaakte doelpunten
sorted_df = standings_df.sort_values(['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'], ascending=[True, True, False, False, False, False])

# Bereken de stand
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag']).cumcount() + 1

# Pas de stand aan voor gelijke standen
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'])['calculated_stand'].transform('min')

# Voeg de berekende stand toe aan de originele DataFrame
standings_df = standings_df.merge(sorted_df[['seizoen', 'speeldag', 'id_ploeg', 'calculated_stand']], on=['seizoen', 'speeldag', 'id_ploeg'], how='left')

# Controleer op discrepanties tussen de berekende en werkelijke stand
discrepancies = standings_df[standings_df['stand'] != standings_df['calculated_stand']]
discrepancies.head(30)


,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten,calculated_goal_verschil,calculated_stand
3,2023,1,3,St-Truidense VV,373,1,1,0,0,1,0,1,3,1,4
4,2023,1,3,Antwerp FC,1,1,1,0,0,1,0,1,3,1,4
5,2023,1,6,R Charleroi SC,22,1,0,1,0,1,1,0,1,0,8
6,2023,1,6,KV Mechelen,25,1,0,1,0,1,1,0,1,0,8
9,2023,1,6,Club Brugge,3,1,0,1,0,1,1,0,1,0,8
10,2023,1,6,OH Leuven,18,1,0,1,0,1,1,0,1,0,8
11,2023,1,12,Cercle Brugge,12,1,0,0,1,0,1,-1,0,-1,13
13,2023,1,12,Standard Luik,16,1,0,0,1,0,1,-1,0,-1,13
18,2023,2,2,St-Truidense VV,373,2,2,0,0,2,0,2,6,2,3
20,2023,2,4,Club Brugge,3,2,1,1,0,2,1,1,4,1,5


In [30]:
filtered_matches = match_results_df[(match_results_df['doelpunten_thuisploeg'] == 5) & (match_results_df['doelpunten_uitploeg'] == 0) | (match_results_df['doelpunten_thuisploeg'] == 0) & (match_results_df['doelpunten_uitploeg'] == 5)]

filtered_matches.head(30)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match
21,2023,3,2023-08-13,16:00:00,4098800,4276,KAS Eupen,3,Club Brugge,0,5,False
356,2022,14,2022-10-23,18:30:00,3851182,16,Standard Luik,35,RSC Anderlecht,5,0,False
376,2022,17,2022-11-11,20:45:00,3851239,18,Oud-Heverlee Leuven,17,RFC Seraing,5,0,False
380,2022,17,2022-11-12,20:45:00,3851244,22,R Charleroi SC,25,KV Mechelen,0,5,False
481,2022,28,2023-03-05,16:00:00,3851121,1,Royal Antwerp FC,25,KV Mechelen,5,0,False
504,2022,31,2023-04-01,18:15:00,3851327,17,RFC Seraing,7,KAA Gent,0,5,False
519,2022,32,2023-04-09,19:15:00,3851351,373,Sint-Truidense VV,31,KV Oostende,5,0,False
695,2021,18,2021-12-11,20:45:00,3598354,17,RFC Seraing,35,RSC Anderlecht,0,5,False
728,2021,22,2022-01-15,16:15:00,3598295,18,Oud-Heverlee Leuven,25,KV Mechelen,5,0,False
782,2021,28,2022-02-19,18:30:00,3598284,19,KV Kortrijk,5381,SV Zulte Waregem,5,0,False
